In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

import torch.nn.functional as F

# Contoh data graf untuk agrikultur
# Node merepresentasikan ladang, edge merepresentasikan hubungan antar ladang (misalnya irigasi)
# Fitur node bisa berupa data seperti kelembaban tanah, suhu, dll.
# Label node bisa berupa hasil panen yang diharapkan.

# Definisi node feature matrix (4 nodes, 3 features per node)
x = torch.tensor([[0.5, 0.3, 0.2],
                  [0.6, 0.1, 0.3],
                  [0.4, 0.4, 0.4],
                  [0.7, 0.2, 0.1]], dtype=torch.float)

# Definisi edge index (hubungan antar node)
edge_index = torch.tensor([[0, 1, 2, 3],
                           [1, 0, 3, 2]], dtype=torch.long)

# Label untuk setiap node (misalnya hasil panen)
y = torch.tensor([1, 0, 1, 0], dtype=torch.long)

# Membuat objek graf
data = Data(x=x, edge_index=edge_index, y=y)

# Definisi model GNN
class GNNModel(torch.nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(3, 16)  # Layer pertama (input feature 3, output feature 16)
        self.conv2 = GCNConv(16, 2)  # Layer kedua (output feature 2 untuk klasifikasi)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Inisialisasi model, optimizer, dan loss function
model = GNNModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.NLLLoss()

# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out, data.y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluasi model
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum()
accuracy = int(correct) / len(data.y)
print(f'Accuracy: {accuracy:.4f}')

Epoch 1, Loss: 0.700269341468811
Epoch 2, Loss: 0.6962003111839294
Epoch 3, Loss: 0.6941051483154297
Epoch 4, Loss: 0.6932327747344971
Epoch 5, Loss: 0.6932733058929443
Epoch 6, Loss: 0.6937888264656067
Epoch 7, Loss: 0.6943231821060181
Epoch 8, Loss: 0.6945958137512207
Epoch 9, Loss: 0.6945645809173584
Epoch 10, Loss: 0.694307267665863
Epoch 11, Loss: 0.6939440965652466
Epoch 12, Loss: 0.693588376045227
Epoch 13, Loss: 0.6933209896087646
Epoch 14, Loss: 0.6931814551353455
Epoch 15, Loss: 0.6931692957878113
Epoch 16, Loss: 0.693251371383667
Epoch 17, Loss: 0.6933764219284058
Epoch 18, Loss: 0.6934919357299805
Epoch 19, Loss: 0.6935596466064453
Epoch 20, Loss: 0.6935632824897766
Epoch 21, Loss: 0.6935087442398071
Epoch 22, Loss: 0.693416953086853
Epoch 23, Loss: 0.6933153867721558
Epoch 24, Loss: 0.6932291388511658
Epoch 25, Loss: 0.6931750178337097
Epoch 26, Loss: 0.6931586265563965
Epoch 27, Loss: 0.6931744813919067
Epoch 28, Loss: 0.6932095289230347
Epoch 29, Loss: 0.6932481527328491

In [6]:
# Import library tambahan untuk Knowledge Graph
from torch_geometric.data import HeteroData

# Contoh data Knowledge Graph
# Menambahkan node baru dari KG (misalnya, node "irigasi")
kg_node_features = torch.tensor([[0.8, 0.2, 0.5]], dtype=torch.float)  # 1 node, 3 fitur
kg_edge_index = torch.tensor([[0, 4],  # Edge dari node ladang ke node irigasi
                              [4, 0]], dtype=torch.long)

# Gabungkan node dan edge dari KG ke graf asli
x = torch.cat([x, kg_node_features], dim=0)  # Tambahkan node KG ke node feature matrix
edge_index = torch.cat([edge_index, kg_edge_index], dim=1)  # Tambahkan edge KG ke edge index

# Update label tensor untuk mencocokkan jumlah node
y = torch.cat([y, torch.tensor([0] * (x.size(0) - y.size(0)), dtype=torch.long)], dim=0)  # Tambahkan label untuk node baru

# Update data graf
data = Data(x=x, edge_index=edge_index, y=y)

# Modifikasi model GNN untuk mendukung node tambahan
class GNNModelWithKG(torch.nn.Module):
    def __init__(self):
        super(GNNModelWithKG, self).__init__()
        self.conv1 = GCNConv(3, 16)  # Layer pertama (input feature 3, output feature 16)
        self.conv2 = GCNConv(16, 2)  # Layer kedua (output feature 2 untuk klasifikasi)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Inisialisasi model, optimizer, dan loss function
model = GNNModelWithKG()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.NLLLoss()

# Training loop tetap sama
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out, data.y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluasi model
model.eval()
_, pred = model(data).max(dim=1)
correct = (pred == data.y).sum()
accuracy = int(correct) / len(data.y)
print(f'Accuracy: {accuracy:.4f}')

Epoch 1, Loss: 0.691676914691925
Epoch 2, Loss: 0.6611071228981018
Epoch 3, Loss: 0.6363438367843628
Epoch 4, Loss: 0.6166200637817383
Epoch 5, Loss: 0.6010981798171997
Epoch 6, Loss: 0.5881333351135254
Epoch 7, Loss: 0.5773992538452148
Epoch 8, Loss: 0.5686701536178589
Epoch 9, Loss: 0.5618618130683899
Epoch 10, Loss: 0.5568491220474243
Epoch 11, Loss: 0.5534699559211731
Epoch 12, Loss: 0.5515027642250061
Epoch 13, Loss: 0.5506738424301147
Epoch 14, Loss: 0.5506765842437744
Epoch 15, Loss: 0.5511984825134277
Epoch 16, Loss: 0.5519502758979797
Epoch 17, Loss: 0.5526894927024841
Epoch 18, Loss: 0.5532344579696655
Epoch 19, Loss: 0.5534674525260925
Epoch 20, Loss: 0.5533307790756226
Epoch 21, Loss: 0.5528168678283691
Epoch 22, Loss: 0.5519565939903259
Epoch 23, Loss: 0.5508078932762146
Epoch 24, Loss: 0.5494437217712402
Epoch 25, Loss: 0.5479426383972168
Epoch 26, Loss: 0.5463811159133911
Epoch 27, Loss: 0.5448269844055176
Epoch 28, Loss: 0.5433355569839478
Epoch 29, Loss: 0.541947305202